In [ ]:
# Key ideas from here (Embedding) https://www.kaggle.com/alexryzhkov/python-keras-nn-residual
# and here (feeding a classification tree from an MLP): https://www.kaggle.com/pourchot/decision-forest-fed-by-neural-network

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt

import optuna
from optuna.samplers import TPESampler

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix,classification_report,log_loss 
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import initializers
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras import activations,callbacks
from tensorflow.keras import layers

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')
train.head()

In [ ]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

# counts number of occurrences by class
sorted(train['target'].value_counts())

In [ ]:
# transform target into 9 columns, one for each class
label_dict = {val:idx for idx, val in enumerate(sorted(train['target'].unique()))}
train['target'] = train['target'].map(label_dict)

lencoder = LabelEncoder()
target = pd.DataFrame(lencoder.fit_transform(train['target']),columns=['target'])
target.head()

#target = train['target']
train.drop(['target'], inplace=True, axis=1)

In [ ]:
sns.countplot(x = 'target', data = target)

In [ ]:
train = train.values
targetMLP = target.values
targetMLP = to_categorical(targetMLP)

In [ ]:
scaler = MinMaxScaler()
# transform data
trainS = scaler.fit_transform(train)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train, targetMLP, test_size = 0.20, 
                                                  stratify = targetMLP, random_state = 2021)

In [ ]:
num_features = 75
num_classes = 9

In [ ]:
model = Sequential([
        layers.Input (shape = (X_train.shape[1],)),
        layers.Embedding(400,8),
        layers.Flatten(),
        Dense(512, kernel_initializer='lecun_normal', activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dense(num_classes, activation = 'softmax')
    ])

model.summary()

In [ ]:
model.compile(loss = CategoricalCrossentropy(),
              optimizer = keras.optimizers.Adam(learning_rate = 2e-4), 
              metrics = 'CategoricalAccuracy')

In [ ]:
es = callbacks.EarlyStopping(
    monitor='val_loss', min_delta = 0.001, patience = 10, verbose = 0,
    mode = 'min', baseline = None, restore_best_weights = True)

plateau = callbacks.ReduceLROnPlateau(
    monitor = 'val_loss', factor = 0.7, patience=5, verbose = 0,
    mode = 'min', min_delta = 0.0001, cooldown = 0, min_lr = 10e-7)

In [ ]:
history = model.fit(X_train, y_train,
            batch_size = 256, epochs = 100, verbose = 2,
            validation_data = (X_val, y_val),
            callbacks=[es,plateau])

In [ ]:
score = model.evaluate(X_val, y_val, verbose = 0)
print('Val loss: {}%'.format(score[0]))
print('Val score: {}%'.format(score[1] * 100))
print("MLP Error: %.2f%%" % (100*(1 - score[1])))

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))
sns.lineplot(x = history.epoch, y = history.history['loss'])
sns.lineplot(x = history.epoch, y = history.history['val_loss'])
ax.set_title('Learning Curve (Loss)')
ax.set_ylabel('Loss')
ax.set_xlabel('Epoch')
ax.legend(['train', 'val'], loc='best')
plt.show()

In [ ]:
X = pd.DataFrame(model.predict(train),columns = sub.columns[1:]) 
X_test = pd.DataFrame(model.predict(test),columns = sub.columns[1:]) 
X_test.head()

In [ ]:
X_tr, X_va, y_tr, y_va = train_test_split(X, target.values, test_size = 0.20, 
                                          stratify = target, random_state = 2021)

In [ ]:
params = {'objective': 'multiclass','num_class' : 9,  'metric': 'multi_logloss', 
              'verbosity' : -1, 'boosting_type' : 'gbdt', 'bagging_freq' : 1}

In [ ]:
def objective(trial):
    
    num_iterations = trial.suggest_int('num_iterations',50,500)
    max_depth = trial.suggest_int('max_depth',3,20)
    num_leaves = trial.suggest_int('num_leaves',10,30)
    learning_rate = trial.suggest_uniform('learning_rate',0.01,0.2)
    subsample = trial.suggest_uniform('subsample',0.1, 0.5)
    feature_fraction = trial.suggest_uniform('feature fraction',0.5, 0.9)
    #min_child_samples = trial.suggest_int('min_child_samples', 1, 110),
    #min_child_weight = trial.suggest_loguniform('min_child_weight' , 1e-5 , 1),
    lambda_l2 = trial.suggest_uniform('lambda_l2',1e-5,20)   
    
    model = LGBMClassifier(**params,
            num_iterations = num_iterations,
            max_depth = max_depth,
            num_leaves = num_leaves,
            learning_rate = learning_rate,
            subsample = subsample,
            feature_fraction = feature_fraction,
            #min_child_samples = min_child_samples,
            #min_child_weight = min_child_weight,
            lambda_l2 = lambda_l2
            )
    
    nll = cross_val_score(model,X_tr,y_tr,scoring = 'neg_log_loss', cv = 5).mean()
    return -1*nll

In [ ]:
sampler = TPESampler(seed=1111)
study = optuna.create_study(direction = 'minimize', sampler = sampler)
study.optimize(objective,n_trials = 1)
print('numbers of the finished trials:' , len(study.trials))
print(study.best_value)
print(study.best_params)

In [ ]:
lgbm = LGBMClassifier(**params, 
                    num_iterations = 93,
                    max_depth = 19,
                    num_leaves = 17,
                    learning_rate = 0.06899061895527038,
                    subsample = 0.10080393597963902,
                    feature_fraction = 0.5942378897566302,
                    #min_child_samples = 27,
                    #min_child_weight = 0.04781667419116532,
                    lambda_l2 = 4.75584202525039)

lgbm.fit(X_tr,y_tr,verbose = False)
preds = lgbm.predict(X_va)

print('Classification report:\n')
print(classification_report(y_va,preds))
sns.heatmap(pd.DataFrame(confusion_matrix(y_va, preds)), annot=True, linewidths=.5, fmt="d")

In [ ]:
sns.countplot(x = 'target', data= pd.DataFrame(preds, columns=['target']))

In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')

sub[sub.columns[1:]] = lgbm.predict_proba(X_test.values)

sub.to_csv("my_submission.csv",index = False)
sub.head()